#### Variables coded in this notebook: 18, 1373, 8345 et seq, 8780 et seq

In [5]:
import os, pandas as pd, re

In [6]:
exportdate = 20180327
projectname = 'repract'

In [7]:
df = pd.read_csv(f'../../data/{exportdate}{projectname}.csv')
df.head(2)

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,v_7039,v_7040,v_7041,...,output_mode,javascript,flash,session_id,language,cleaned,ats,datetime,date_of_last_access,date_of_first_mail
0,106,0,no tester,Completed after break (32),2138658,NotShown,-1,NotShown,NotShown,0,...,HTML,NotShown,NotShown,3bb21c1b318e2f6b87557566bdd6b4d9,English,Not cleaned,1515411510,2018-01-08 11:38:30,2018-01-08 13:07:14,0000-00-00 00:00:00
1,131,0,no tester,Completed (31),2138658,NotShown,3805,NotShown,NotShown,NotShown,...,HTML,NotShown,NotShown,fc38f6556787a459c2cc604abf799448,English,Not cleaned,1515667019,2018-01-11 10:36:59,2018-01-11 11:40:24,0000-00-00 00:00:00


In [8]:
basedir = '../../data/freetext'
freetextfiles = os.listdir(basedir)
dfs = {file[:-4]:pd.read_csv(f'{basedir}/{file}') for file in freetextfiles}

In [9]:
dfs.keys()

dict_keys(['.DS_S', 'v_11', 'v_1373', 'v_16', 'v_18', 'v_19', 'v_6', 'v_8345etseq', 'v_8780etseq'])

Leaving this here temporarily merely for reference!

In [120]:
def add_unique_code(df, func, varname):
    df[f'{varname}_coded'] = func(df[varname])
    return df

In [79]:
codedir = '../../analysis/freetext'
def write_coded(df, varname):
    filepath = f'{codedir}/{varname}_coded.csv'
    df.to_csv(f'{codedir}/{varname}_coded.csv', index=False)
    print(f'File stored at {filepath}.')

For long entries, we actually want to enable multiple tagging, therefore, we want to produce a long-form DF

The stuff below is just for the structure - TODO: fill in sensible info...

In [46]:
def code_var_long(df, index_col, coding_col, partial_df=None):
    # assumes input df sorted by index col
    new_df = partial_df if partial_df is not None else pd.DataFrame(columns=[
        index_col, f'{coding_col}_code'])
    min_index = max(partial_df[index_col]) if partial_df is not None else 0
    for row in df[[index_col, coding_col]].iterrows():
        if row[0] > min_index:
            print(row[1][1])
            codes = input('Specify codes separated by commas (Code A,Code B,). '
                      +'Enter X to quit and return partial frame.\n'
                     ).split(',')
            print('\n')
            if codes[0] == 'X':
                write_coded(newdf, f'{coding_col}_code')
                return new_df.reset_index().drop('index', axis=1)
            for code in codes:
                new_df = new_df.append(pd.DataFrame({index_col:[row[1][0]],f'{coding_col}_code':[code]}))
    write_coded(newdf, f'{coding_col}_{min_index}-{row[0]}_code')
    return new_df.reset_index().drop('index', axis=1)

Potential Categories:
- Practical Problem (Relevance)
- Sensible Solution (Plausibility)
- Astute Approach (Originality)
- NotAnswered (String with no true content)

In [146]:
# Relevance
x = 0
for idx, r in enumerate(dfs['v_8345etseq']['reasoning']):
    if (re.search(
        'practi',r)
        ):
        print(idx, r)
        x +=1 
x

8 These practices and artifacts are essential for our work, and are broadly applicable. Their ubiquity often means they are subject to unstated assumptions of their utility. Understanding how others have used them could provide insights into our own work. 
20 In my experience, framing requirements as a constraint for a creative problem solving process yields better solutions. Any help with establishing and improving such practices will likely help the industry provide better solutions. 
54 This research addresses a very relevant issue for the industry, it may lead to important practical applications. 
66 This dilemma occurs constantly in practice: time vs. quality.
77 In my opinion we need to share and exchange more information related to the practices that we have been using in order to increase our performance and maturity level.
92 Will enable upfront proof and a way to measure in practice. Confidence to the team and the client.
107 There is a tension between RE rigour and agile; un

9

In [123]:
def match(series, regex, idx_only=False):
    if idx_only:
        return [idx for idx, x in enumerate(series) if re.search(regex, x)]
    else:
        return [(idx, x) for idx, x in enumerate(series) if re.search(regex, x)]

def find_unmatched(series, regexes):
    series_idx = {idx for idx, x in enumerate(series)}
    matched_idx = set()
    for regex in regexes:
        matched_idx = matched_idx.union(set(match(series, regex, True)))
    return sorted(list(series_idx.difference(matched_idx)))

In [150]:
regexes = [
    # Relevance
    '[Pp]roblem|[Cc]halleng|[Ee]xperienc|[Rr]elevan|[Ee]ssential|[Cc]ritical|[Cc]rucial|[Ii]mporta|[Ii]ssue(?!s)|[Cc]oncern|[Dd]ifficult|[Nn]eed(?!s)|(?:^|\W)[Ww]e\W|[Ff]undamental|[Dd]ilemma',
    # Plausibility
    '[Cc]ould|[Mm]ight|[Hh]elp(s|ful)?(?!\w)|[Ii]mproves?(?!m)|(?<!\sto\s)better',
    # Originality
    'literature|gap|interesting'
]
len(find_unmatched(dfs['v_8345etseq']['reasoning'], regexes))

37

In [148]:
for idx, x in enumerate(dfs['v_8345etseq']['reasoning']):
    if idx in find_unmatched(dfs['v_8345etseq']['reasoning'], regexes):
        print(x)

.
A learning-by-example method is one of the most efficient tools allowing for decreasing the uncertainty in the future.
Benchmarks are the start of everything.
A many more automatical methods shall support and ease the work of RE. Especially for standardised requirements, security, safety, traceability analysis,etc. 
non-functional reqs have the biggest impact on architecture and may not be left aside 
it will give me good reasones for RE that managers will understand
aren't they connected?
I'm working in agile development environments. This items affects my work significantly.
Because of the conflict between complete requirements and fast requirements.
A lot of regulations are contradictory, it is good to have an overview for decision making
I think that communication in general is a success factor of a project; good communication drives to the right goals.
Green field happens from time to time; incremental improvements are very common. Change requests occur frequently
Non-functional

Procedure:
- Refine Regex to the point where further refinement would lead to drastical overfitting
- Assign Reasoning Tags via Automation
- Find Answers with no Tags and Assign Manually
- Verify all tags